# 랜덤 포레스트

### 정형 데이터
어떤 구조로 되어 있다는 뜻, CSV나 데이터 베이스, 엑셀에 저장하기 쉽다.

### 비정형 데이터
텍스트 데이터, 디지털 음악, 사진 등이 비정형 데이터에 속한다.

정형 데이터를 다루는 데 가장 뛰어난 성과를 보이는 알고리즘이 앙상블 학습이다.
이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있다.

비정형 데이터는 어떤 알고리즘을 사용해야할까?
신경망 알고리즘을 주로 사용한다.

### 랜덤 포레스트

랜덤 포레스트는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용된다.
랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다.
그리고 각 결정 트리의 예측을 사용하여 최종 예측을 만든다.

랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 이데이터를 만드는 방식이 독특하다.
우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터륾 만든다.

1000 개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣는다.
이런식으로 계속 해서 100개를 가방에서 뽑으면 중복된 샘플을 뽑을 수 있다.
이렇게 만들어진 샘플을 부트스트랩 샘플이라고 부른다.

또한 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른다음 이 중에서 최선의 분할을 찾는다.

분류 모델인 RandomForestClassifier 는 기본적으로 전체 특성 개수의 제곱근 만큼의 특성을 선택한다.

즉 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용한다.

다만 회귀 모델인 RandomForestRegressor 는 전체 특성을 사용한다.

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련한다.

그 다음 분류 시에는 각 트리의 클래스별 확률을 평군하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다.
회귀일 대는 단순히 각 트리의 예측을 평균한다.


랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.
종종 기본 매개변수 설정 만으로도 아주 좋은 결과를 낸다.



In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
# wine.head()
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

cross_validate() 함수를 사용하여 교차 검증을 수행,
RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1 로 지정하여 모든 CPU 코어를 사용하는 것이 좋다.

return_train_score 매개변수를 True로 지정하면 검증 점수뿐 아니라 훈련 세트에 대한 점수도 같이 반환된다.
훈련세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는 데 용이하다.

In [12]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


랜덤 포레스트는 결정 트리의 앙상블이기때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다.

criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등이다. 또한 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산한다.

랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.


In [14]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


각각 [알코올 도수, 당도, pJ] 였는데, 두번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승했다.

이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다.
그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.

그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.
이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.

RandomForestClassifier에는 기능이 하나 더있는데, 자체적으로 모델을 평가하는 점수를 얻을 수 있다.
랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리를 훈련한다.
이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있는데 이를 OOB (Out Of Bag) 라 한다.

이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정트리를 평가가 가능하다.



In [15]:
rf = RandomForestClassifier(oob_score=True, n_jobs = -1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻었다.
OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용이 가능하다.


# 엑스트라 트리
엑스트라 트리는 랜덤 포레스트와 매우 비슷하게 동작한다. 기본적으로 100개의 결정 트리를 훈련한다.
랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원한다.
또한 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용한다.

랜덤 포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는 점이다.
즉각 결정 트리를 만들 때 전체 훈련세트를 사용한다.
대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.
사이킷런에서 제공하는 엑스트라 트리는 ExtraTreesClassifier이다.


In [17]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻었다. 특성이 많지 않아 두 모델의 차이가 크지 않다.
보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야한다.
랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.

엑스트라 트리도 랜덤 포레스트와 마찬가지로 특성 중요도를 제공한다.
순서는 [알코올 도수, 당도, pH]인데, 결과를 보면 엑스트라 트리도 결정 트리보다 당도에 대한 의존성이 작다.


In [18]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


엑스트라 트리의 회귀버전은 ExtraTreeRegressor 클래스이다.


# 그레이디언트 부스팅

그레이디언트 부스팅 gradient boosting은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방식이다.

사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정트리를 100개 사용한다.
깊이가 얕은 결정 트리를 사용하기 떄문에 과대 적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

그레이디언트란 경사 하강법을 사용하여 트리를 앙상블에 추가한다. 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용한다.

경사 하강법은 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정이다.

이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것이다.

그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동한다.
또 학습률 매개 변수로 속도를 조절한다.


In [19]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(
    gb, train_input, train_target,
    return_train_score=True, n_jobs=-1
)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.
학습률을 증가시키고 트리의 개수를 늘리면 성능이 향상된다.

In [20]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 500개로 5배나 늘렸지만 과대적합을 잘 억제하고 있다.
학습률 learning_rate의 기본값은 0.1이다.
그레이디언트 부스팅도 특성 중요도를 제공한다.
결과에서 볼 수 있듯이 그레이디언트 부스팅이 랜덤 포레스트보다 더 당도에 집중한다.

In [23]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
gb.score(test_input, test_target)

[0.15872278 0.68010884 0.16116839]


0.8707692307692307

트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample 있다.
이 매개변수의 기본값은 0.1로 전체 훈련 세트를 사용한다.
하지만 subsample이 1보다 작으면 훈련 세트의 일부를 사용한다.
이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷하다.

일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있다.

하지만 순서대로 트리를 추가하기때문에 훈련속도가 느리다.

즉 GradientBoostingClassifier에는 n_jobs 매개변수가 없다.
그레이디언트 부스팅의 회귀 버전은 GradientBoostingRegressor이다.



### 히스토그램 기반 그레이디언트 부스팅

히스토그램 기반 그레이디언트 부스팅은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다.
히스토그램 기반 그레이디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 나눈다.

따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용된다.

HisGraidientBoostingClassifier을 사용하고, 기본 매개변수에서 안정적인 성능을 얻을 수 있다.
해당 클래스에서는 트리의 개수를 지정하는 대신에 max_iter를 사용하여 부스팅 반복 횟수를 지정한다.

와인 데이터셋에 HistGradientBoostingClassifier 클래스를 적용하자.
사이킷런의 히스토그램 기반 그레이디언트 부스팅은 아직 테스트 과정중이다.


In [24]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면서 그레이디언트 부스팅보다 더 높은 성능이다.
히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계한하기 위해 permutation_importance 함수를 사용하자.

이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다.

훈련 세트뿐만 아니라 테스트세트에도 적용이 가능하고, 사이킷런에서 제공하는 추정기 모델에 모두 사용이 가능하다.

히스토그램 기반 그레이디언트 부스팅 모델을 훈련하고 훈련세트에서 특성 중요도를 계산해보자.
n_repaets 매개변수는 랜덤하게 섞을 횟수를 지정한다.
기본값은 5이다.

In [26]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(
    hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1
)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]
[0.08876275 0.23438522 0.08027708]


permutation_importance 함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준편차를 담고 있다.
평균을 출력해보면 랜덤 포레스트와 비슷한 비율이다.

In [28]:
result = permutation_importance(hgb, test_input, test_target,
                                 n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있다.
이런 분석을 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상이 가능하다.


In [29]:
hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost를 사용

In [30]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


히스토그램 기반 그레이디언트 부스팅 라이브러리 마이크로소프트의 LightGBM


In [31]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [ ]:
print(np.mean(scores['train_score']),np.mean(scores['test_scores']))